<a href="https://colab.research.google.com/github/kshah777/CSMC512PCL/blob/main/Python_Script_PCL_Detection3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Section : Import all required Packages

In [164]:
# Used to pull data from google drie
from google.colab import auth
from googleapiclient.discovery import build
import io
from io import FileIO
from googleapiclient.http import MediaIoBaseDownload

# Used for reading csv file
import pandas

# Module for contraction expansion
!pip install contractions
import contractions

#Module will be used for punctuation Removal
from string import punctuation

#Tokenizer 
!pip install nltk
import nltk
nltk.download('punkt')

# Import stopword list
nltk.download('stopwords')

# Import Lemmatizer
nltk.download('wordnet')

# Used for random selection in subsampling
import random

#Import TF-IDF
!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Import BERT Things
!pip install transformers
from transformers import BertTokenizer
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertConfig
from transformers import get_linear_schedule_with_warmup
import numpy as np

# Import Neural network modules
from torch import nn
import torch.nn.functional as F
from transformers import AdamW
from torch import optim


print("Finished Importing")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Finished Importing


Set Hyper-Parameters

In [165]:
#Data-PrePre-processing parameters
remove_non_strings = True # Make this true please

#Data-Pre-processing parameters
expand_contractions = True
remove_punctuation = True
make_lowercase = True  #Bert and I think TFIDF is uncased, so regardless of this input to it will be lowercase.
#tokenize = True      Not actually a choice
remove_stop_words = True
lemmatization = True
#NOTE, if remove_stop_words or lemmatization = TRUE, tokenize must also equal TRUE

#Post Pre-processing data splitting parameters:
remove_too_long = True # This will remove values that are over 512 character and therefore would just be truncated later
random_seed = 0       # If Random seed is set to zero, seed will be automically generated
proportion_train_data = 0.8

# Full Model Paramters
TF_IDF = True
BERT = True

#TF-IDF Parameters
tfidf_maxfeatures = 384   #Note, Bert will provide a vecotr of length 768
                          #Note 2: 2 tfidf vectors made (pcl, nonpcl), so max features * 2 is true length of tfidf vector

# BERT Parameters
batch_size_param = 16
max_len = 256  #Recommend keeping bellow 400, Allows for BERT Token characters to reach max wtih special characters 512
truncate = True

#Neural Network parameters
num_epochs = 15
learning_rate = 0.003
batch_size_nn = 8


#Select Gpu
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



No GPU available, using the CPU instead.


Data Import from Google Drive

In [166]:
# Download File from Google Drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Import Data File
file_id = '159fu3B8BaoKzXMbTHFDYXSHy1AuVA1f7'  # Training file on the Google Drive
downloaded = io.FileIO("dontpatronizeme_pcl.tsv", 'w')
request = drive_service.files().get_media(fileId=file_id)
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download {}%.".format(int(status.progress() * 100)))

Download 100%.


Read Data CSV, find indeces that contain non-string and remove. Make list of all strings that are above current max lenth

In [167]:
#Reads data set, skipping header rows starting with the first line (fixed issue where rows were combining by adding other delimiter)
input_df = pandas.read_csv("dontpatronizeme_pcl.tsv", delimiter='\t|\\t', header=None, names=['trash', 'trash2', 'trash3', 'text', 'label'], skiprows = 4)

#Create lists containing the text and labels (indces will line up)
input_instances = input_df.text.values
input_labels = input_df.label.values

#Convert to normal lists because I am more comfortable with them
input_labels = input_labels.tolist()
input_instances = input_instances.tolist()

#Print initial number of instances
print("Initial Number of Instances:", len(input_instances))
print("Same count labels vs text instances:", len(input_labels)==len(input_instances), "\n")

# Find out which indeces are too long or are empty/dont contain strings (dataset has some noise)
indeces_delete_Empty = []
indeces_delete_tooLong = []
for i in range(len(input_instances)):
    if type(input_instances[i]) is not str:
        indeces_delete_Empty.append(i)
    # elif len(input_instances[i]) > max_len:
    #     indeces_delete_tooLong.append(i)  
print("Indeces that are empty/do not contain strings:", len(indeces_delete_Empty))
# print("Indeces that are currently too long (length > {}):".format(max_len), len(indeces_delete_tooLong))

# Remove non strings (acording to hyperparameter)
if remove_non_strings:
  for i in reversed(indeces_delete_Empty):
    input_labels.pop(i)
    input_instances.pop(i)
    print("Deleted All Empty or non String entries")

#Print  number of instances after prepre-processing
print("\nPost-prepre processing Number of Instances:", len(input_instances))
print("Same count labels vs text instances:", len(input_labels)==len(input_instances), "\n")

Initial Number of Instances: 10636
Same count labels vs text instances: True 

Indeces that are empty/do not contain strings: 1
Deleted All Empty or non String entries

Post-prepre processing Number of Instances: 10635
Same count labels vs text instances: True 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Pre processing steps dependent on hyper paramters aboce

In [168]:
# The following webstie was heavily referenced for the text preprocessing:
# https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/

# Also referenced geeksforgeeks.org for some of these processes
# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# https://www.geeksforgeeks.org/python-lemmatization-with-nltk/

# If hyper parameter, expand contraction

unaltered_instances = np.copy(input_instances)

if expand_contractions:
  print("Removing Contractions")
  texts_altered = 0
  text_fix = " "
  for i in range(len(input_instances)):
    text = input_instances[i]
    text_fix = contractions.fix(text)  
    if text_fix != text:
      texts_altered += 1
      input_instances[i] = text_fix
  print("Contractions Contained and Expanded in {} of the text excerpts\n".format(texts_altered))

# If hyper parameter, remove punctuation
if remove_punctuation:
  print("Removing Punctuation")
  for i in range(len(input_instances)):
    text = "".join([i for i in input_instances[i] if i not in punctuation])
    input_instances[i] = text
  print("Finished Removing Punctuation\n")

# If hyper parameter, make all lowercase
if make_lowercase:
  print("Making Everything Lowercase")
  for i in range(len(input_instances)):
    text = input_instances[i]
    input_instances[i] = text.lower()
  print("Finished Making Things Lowercase\n")

# Always tokenize
tokenize = True
if tokenize:
  print("Tokenizing")
  for i in range(len(input_instances)):
    text = input_instances[i]
    text_token = nltk.word_tokenize(text)
    input_instances[i] = text_token
  print("Finished Tokenizing. Sentences Split to Arrays\n")


# If hyper parameter, remove stop words (Maybe look at altering stop word list?)
if remove_stop_words:
  print("Removing Stop Words")
  print("The following Stop Words will be removed")
  stops = nltk.corpus.stopwords.words('english')
  for word_ind in range(0,len(stops), 5):
    if(len(stops) - word_ind > 5):
      print("{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}".format(stops[word_ind],stops[word_ind+1],stops[word_ind+2],stops[word_ind+3],stops[word_ind+4]))
    else:
      print(stops[word_ind], sep= "\t")
  for i in range(len(input_instances)):
    text = input_instances[i]
    input_instances[i]= [j for j in text if j not in stops]
  print("Stop Words Removed\n")
      
# If hyper parameter, Lemmatize
if lemmatization:
  print("Lemmatizing")
  lemmatizer = nltk.wordnet.WordNetLemmatizer()
  count= 0
  for i in range(len(input_instances)):
    text = input_instances[i]
    input_instances[i] = [lemmatizer.lemmatize(word) for word in text]
  print("Lemmatizing Finished\n")


    

Removing Contractions
Contractions Contained and Expanded in 516 of the text excerpts

Removing Punctuation
Finished Removing Punctuation

Making Everything Lowercase
Finished Making Things Lowercase

Tokenizing
Finished Tokenizing. Sentences Split to Arrays

Removing Stop Words
The following Stop Words will be removed
i       	me      	my      	myself  	we      
our     	ours    	ourselves	you     	you're  
you've  	you'll  	you'd   	your    	yours   
yourself	yourselves	he      	him     	his     
himself 	she     	she's   	her     	hers    
herself 	it      	it's    	its     	itself  
they    	them    	their   	theirs  	themselves
what    	which   	who     	whom    	this    
that    	that'll 	these   	those   	am      
is      	are     	was     	were    	be      
been    	being   	have    	has     	had     
having  	do      	does    	did     	doing   
a       	an      	the     	and     	but     
if      	or      	because 	as      	until   
while   	of      	at      	by      	for     

This Section will now seperate the data into arrays for PCL containing and non PCL containing Data. Non PCL randomly samples to create equal arrays

In [169]:
#Make sure data lengths didnt change following pre-processing
print("Text after Pre Proccessing\n------------------")
print("Post-prepre processing Number of Instances:", len(input_instances))
print("Same count labels and text instances?", len(input_labels)==len(input_instances), "\n")

#Check for empty strings (shouldn't be any, but who know)
# and check for strings that are too long(just remove them, smaller number now)
indeces_delete_Empty = []
indeces_delete_tooLong = []

max_obs_len =0
for i in range(len(input_instances)):
    max_obs_len = max(max_obs_len, len(input_instances[i]))
    if len(input_instances[i]) == 0:
        indeces_delete_Empty.append(i)
    elif len(input_instances[i]) > max_len:
        indeces_delete_tooLong.append(i)  
print("Indeces that are empty/do not contain strings:", len(indeces_delete_Empty))
print("Indeces that are currently too long (length > {}):".format(max_len), len(indeces_delete_tooLong))


print("Max observed length is...{}".format(max_obs_len))

unaltered_instances = unaltered_instances.tolist()
#Delete empty strings and long strings according to parameters
if remove_non_strings:
  for i in reversed(indeces_delete_Empty):
    input_labels.pop(i)
    input_instances.pop(i)
    unaltered_instances.pop(i)
    print("Deleted All Empty or non String entries")
if remove_too_long:
  for i in reversed(indeces_delete_tooLong):
    input_labels.pop(i)
    input_instances.pop(i)
    unaltered_instances.pop(i)
  print("Deleted All Strings Over {} character\n".format(max_len))

print("Text after Deletions\n------------------")
print("Post-deletion Number of Instances:", len(input_instances))
print("Same count labels and text instances?", len(input_labels)==len(input_instances), "\n")

# Create list of indeces with pcl (3-4), no pcl (0-1), and count of abiguoius ones(2)
pcl_indeces = []
nonpcl_indeces = []
count_ambiguous= 0
for i in range(len(input_labels)):
    label = input_labels[i]
    if label>=3:
        pcl_indeces.append(i)
    elif label<2:
        nonpcl_indeces.append(i)
    else:
        count_ambiguous +=1

# Print diagnostic info to make sure  things acting as expected 
print("PCL occurencess:", len(pcl_indeces))
print("Non-PCL occurences:", len(nonpcl_indeces))
print("Removed for abiguity:", count_ambiguous)
print("Pcl+Non-Pcl+ambiguous = total instances?", (len(pcl_indeces)+len(nonpcl_indeces)+count_ambiguous == len(input_instances)), "(Expected to be true)")

# Set random seed if provided
if random_seed:
  print("Random seed is {}".format(random_seed))
  random.seed(random_seed)

#shuffle the pcl_indeces
random.shuffle(pcl_indeces)


#randomly sample equal proportion of non-pcl language
nonpcl_indeces_shortened = []
nonpcl_indeces_shortened = random.sample(nonpcl_indeces, len(pcl_indeces))
print("\nNon-PCL Occurneces After Random Sample:", len(nonpcl_indeces_shortened))


# Combine the Items (which are currently arrays) back into sentences, if they were tokenized
if tokenize:
  for i in range(len(input_instances)):
    text = input_instances[i]
    input_instances[i] = " ".join(text)

  print("Sentences are back to strings")


Text after Pre Proccessing
------------------
Post-prepre processing Number of Instances: 10635
Same count labels and text instances? True 

Indeces that are empty/do not contain strings: 0
Indeces that are currently too long (length > 256): 2
Max observed length is...499
Deleted All Strings Over 256 character

Text after Deletions
------------------
Post-deletion Number of Instances: 10633
Same count labels and text instances? True 

PCL occurencess: 852
Non-PCL occurences: 9632
Removed for abiguity: 149
Pcl+Non-Pcl+ambiguous = total instances? True (Expected to be true)

Non-PCL Occurneces After Random Sample: 852
Sentences are back to strings


Will now split the groups into test and train data, ratio specified by user

In [170]:
#calculate cutoff value which will seperate user defined value to test/train
test_index_cutoff = int(len(pcl_indeces)*proportion_train_data)
print("{:.1%} cutoff at index: {}".format(proportion_train_data, test_index_cutoff))


#Create train/test lists for instances and labels using the  index lists
train_instances = []
train_labels = []
train_instances_unaltered = []
test_instances = []
test_labels = []
test_instances_unaltered = []

for i in range(len(pcl_indeces)):
    pcl_index = pcl_indeces[i]
    nonpcl_index = nonpcl_indeces_shortened[i]
    if i < test_index_cutoff:
        train_instances.append(input_instances[pcl_index])
        train_instances_unaltered.append(unaltered_instances[pcl_index])
        train_labels.append(input_labels[pcl_index])
        train_instances.append(input_instances[nonpcl_index])
        train_instances_unaltered.append(unaltered_instances[nonpcl_index])
        train_labels.append(input_labels[nonpcl_index])
    else:
        test_instances.append(input_instances[pcl_index])
        test_instances_unaltered.append(unaltered_instances[pcl_index])
        test_labels.append(input_labels[pcl_index])
        test_instances.append(input_instances[nonpcl_index])
        test_instances_unaltered.append(unaltered_instances[nonpcl_index])
        test_labels.append(input_labels[nonpcl_index])

#Print Diagnostic info to make sure things are behaving as expected
print("\nTest Instances:",len(test_instances))
print("Unaltered Test Instances: {}".format(len(test_instances_unaltered)))
print("Test Label:", len(test_labels))
print("Train Instances:",len(train_instances))
print("Unaltered Train Instances: {}".format(len(train_instances_unaltered)))
print("Train Label:", len(train_labels))

print("\nExpected Combined Length:", len(pcl_indeces)+len(nonpcl_indeces_shortened))
print("Actual Combined Length:", len(train_instances)+ len(test_instances))

print("\nTransform labels to 1 and 0\n1=pcl, 0 = no pcl ")
#Replace 1->0, 3and 4 to 1. Only other possible input is 0 which should remain
for i in range(len(train_labels)):
    if train_labels[i] == 1:
        train_labels[i] = 0
    elif train_labels[i] == 3:
        train_labels[i] = 1
    elif train_labels[i] == 4:
        train_labels[i] = 1
for i in range(len(test_labels)):
    if test_labels[i] == 1:
        test_labels[i] = 0
    elif test_labels[i] == 3:
        test_labels[i] = 1
    elif test_labels[i] == 4:
        test_labels[i] = 1

print("\n\nFinal Diagnostic Information After seperation:")
print("Are train label and instances the same length? {}\t Length is {}".format(len(train_instances)==len(train_labels), len(train_instances)))
print("Are test label and instances the same length? {}\t Length is {}".format(len(test_instances)==len(test_labels), len(test_instances)))
print("Proportion of PCL in train: {}".format(train_labels.count(1)/len(train_labels)))
print("Proportion of PCL in test: {}".format(test_labels.count(1)/len(test_labels)))



80.0% cutoff at index: 681

Test Instances: 342
Unaltered Test Instances: 342
Test Label: 342
Train Instances: 1362
Unaltered Train Instances: 1362
Train Label: 1362

Expected Combined Length: 1704
Actual Combined Length: 1704

Transform labels to 1 and 0
1=pcl, 0 = no pcl 


Final Diagnostic Information After seperation:
Are train label and instances the same length? True	 Length is 1362
Are test label and instances the same length? True	 Length is 342
Proportion of PCL in train: 0.5
Proportion of PCL in test: 0.5


TF - IDF Implementation          Adapted from: https://sanjayasubedi.com.np/machinelearning/nlp/text-classification-with-sklearn/ and https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.YXgb1L_MJH4

In [171]:
# # Check if when training you are supposed to only include postive(PCL containing) textx

if TF_IDF:
  #Creates tfidf scores for the train values, will be called later

  #Initialize Vectorizer for Tfidf which will choose 1000 most informative features
  vectorizer_pcl = TfidfVectorizer(max_features = tfidf_maxfeatures, use_idf=True)
  vectorizer_nopcl = TfidfVectorizer(max_features = tfidf_maxfeatures, use_idf=True)

  tfidf_pcl_train = []
  tfidf_nopcl_train = []
  for i in range(len(train_labels)):
    if train_labels[i] == 1:
      tfidf_pcl_train.append(train_instances[i])
    else:
      tfidf_nopcl_train.append(train_instances[i])
  train_vectorized_pcl = vectorizer_pcl.fit_transform(tfidf_pcl_train)
  train_vectorized_nopcl = vectorizer_nopcl.fit_transform(tfidf_nopcl_train)

  #This is example of creating normalarray which will need to be appended to BERT array before NN
  #print(vectorizer.transform([test_instances[0]]).toarray())
  print("TFIDF Matrix Created")
else:
  print("TFIDF Not created due to earlier parameters")


if TF_IDF:   #Behind if statement because only want to run if TF-IDF being created
  # Create Array of tfidf result for each train instances:
  train_tfidf_pcl = vectorizer_pcl.transform(train_instances).toarray()
  train_tfidf_nopcl = vectorizer_nopcl.transform(train_instances).toarray()

  #COMBINE THOSE 2 together to be train_tfidf
  train_tfidf = []
  for i in range(len(train_tfidf_pcl)):
    to_append = np.concatenate((train_tfidf_pcl[i], train_tfidf_nopcl[i]), axis=None)
    train_tfidf.append(to_append)
  print("TFIDF Results For Train Data Complete")
  print("Dimension is {} x {}".format(len(train_tfidf), len(train_tfidf[0])))

  # Make Test array right now too, cause why not
  test_tfidf_pcl = vectorizer_pcl.transform(test_instances).toarray()
  test_tfidf_nopcl = vectorizer_nopcl.transform(test_instances).toarray()

  #COMBINE THOSE 2 together to be test_tfidf
  test_tfidf = []
  for i in range(len(test_tfidf_pcl)):
    to_append = np.concatenate((test_tfidf_pcl[i], test_tfidf_nopcl[i]), axis=None)
    test_tfidf.append(to_append)

  print("TFIDF Results For Test Data Complete")
  print("Dimension is {} x {}".format(len(test_tfidf), len(test_tfidf[0])))

TFIDF Matrix Created
TFIDF Results For Train Data Complete
Dimension is 1362 x 768
TFIDF Results For Test Data Complete
Dimension is 342 x 768


BERT For Classification Implementation: Adapted from https://mccormickml.com/2019/07/22/BERT-fine-tuning/#12-installing-the-hugging-face-library, also took notes from Amy Olex's Bert Implementation

This Section Finds and sets the true max length used in the classification.


In [172]:
if BERT:
  #Set train instances and test instances to the unmodified versions (no preproccessing has occured)
  train_instances = train_instances_unaltered
  test_instances = test_instances_unaltered

  # Load the BERT tokenizer. Will be used to Re-tokenize data
  print('Loading BERT tokenizer...')
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


  # ''' Shows how Bert Tokenized
  # # Print the original sentence.
  # print(' Original: ', train_instances[0])
  # # Print the sentence split into tokens.
  # print('Tokenized: ', tokenizer.tokenize(train_instances[0]))
  # # Print the sentence mapped to token ids.
  # print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_instances[0])))
  # '''

  max_obs_len = 0
  # Check all train data and find highest token character
  for text in train_instances:
      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(text, add_special_tokens=True)
      # Update the maximum sentence length.
      max_obs_len = max(max_obs_len, len(input_ids))

  for text in test_instances:
    # Check all train data and find highest token character
      input_ids = tokenizer.encode(text, add_special_tokens=True)
      # Update the maximum sentence length.
      max_obs_len = max(max_obs_len, len(input_ids))


  print('Max Observed sentence length: ', max_obs_len)

  #Select the minimum of the maximum lengths. BERT will truncate any sentances longer than this. 
  max_len = min(max_obs_len, max_len)


Training Tokenization:

In [173]:
if BERT:
  print("Tokenize all of the training sentences and map the tokens to thier word IDs.")
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  train_input_ids = []
  train_attention_masks = []

  # For every sentence...
  for text in train_instances:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length'
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          text,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = max_len,           # Pad & truncate all sentence
                          pad_to_max_length=True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.

                    )
      
      # Add the encoded sentence to the list.    
      train_input_ids.append(encoded_dict['input_ids'])
      # And its attention mask (simply differentiates padding from non-padding).
      train_attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.

  train_input_ids = torch.stack(train_input_ids).squeeze()
  train_attention_masks = torch.stack(train_attention_masks).squeeze()
  train_label_tensor = torch.tensor(train_labels)
  
  #Make train dataset
  #train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

  # Check dimensionality (should be same)
  # print(train_input_ids.shape)
  # print(train_attention_masks.shape)
  # print(train_label_tensor.shape)

  train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_label_tensor)

  # ''' Print to see what tensor looks like
  # # Print sentence 0, now as a list of IDs.
  # print('Original: ', train_instances[0])
  # print('Token IDs:', train_input_ids[0])
  # '''

Test Tokenizer

In [174]:
if BERT:
  print("Tokenize all of the testing sentences and map the tokens to thier word IDs.")
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  test_input_ids = []
  test_attention_masks = []

  # For every sentence...
  for text in test_instances:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length'
      #   (6) Create attention masks for [PAD] tokens.
      test_encoded_dict = tokenizer.encode_plus(
                          text,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = max_len,           # Pad & truncate all sentences.
                          truncation = True,
                          pad_to_max_length=True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.

                    )
      
      # Add the encoded sentence to the list.    
      test_input_ids.append(test_encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      test_attention_masks.append(test_encoded_dict['attention_mask'])

  # Convert the lists into tensors.

  test_input_ids = torch.stack(test_input_ids).squeeze()
  test_attention_masks = torch.stack(test_attention_masks).squeeze()
  test_labels_tensor = torch.tensor(test_labels)

  #Make test dataset
  test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels_tensor)

  # '''Test the Output
  # # Print sentence 0, now as a list of IDs.
  # print('Original: ', test_instances[0])
  # print('Token IDs:', test_input_ids[0])
  # '''

Create Data Loader for train and test datasets

In [175]:
if BERT:
  #Create the DataLoaders for our training and validation sets.
  #We'll take training samples in random order. 
  print("Making Train dataloader")
  train_dataloader = DataLoader(
              train_dataset,  # The training samples.
              sampler = RandomSampler(train_dataset), # Select batches randomly
              batch_size = batch_size_param # Trains with this batch size.
          )

  print("Making Test dataloader")
  # For validation the order doesn't matter, so we'll just read them sequentially.
  validation_dataloader = DataLoader(
              test_dataset, # The validation samples.
              sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
              batch_size = batch_size_param # Evaluate with this batch size.
          )

Loading Bert and setting parameters

In [176]:
if BERT:
  # Load BertForSequenceClassification, the pretrained BERT model with a single 
  # linear classification layer on top. 

  print("Loading BERT")
  model = BertModel.from_pretrained(
      "bert-base-uncased" # Use the 12-layer BERT model, with an uncased vocab.
  )

  # Tell pytorch to run this model on the GPU.
  model.to(device)

#Set seeds if random seed it set
if random_seed:
  np.random.seed(random_seed)
  torch.manual_seed(random_seed)
  torch.cuda.manual_seed_all(random_seed)

Get BERT embeddings for train data
BERT embedding is average vector for sentence. Result has length 768.

In [177]:
if BERT:
  model.eval()

  #print(b_input_ids[0])

  train_word_embed_eachinstance = []
  count = 0
  print("There will be {} batches".format(len(train_dataloader)))
  for batch in train_dataloader:
    if count%5==0 and count!=0:
      print("On batch number {}".format(count))
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model_bert not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        #train_cls_head.append(outputs[1])
        #train_word_embed_eachinstance.append(outputs[0])
        #print(len(outputs[0]))
        for sentence in outputs[0]:
          #print(len(sentence))
          sentence_embed = []
          for embed_index in range(len(sentence[0])):
            index_total = 0
            for word in sentence:
              index_total += word[embed_index]
            sentence_embed.append(index_total / len(word))
          train_word_embed_eachinstance.append(sentence_embed)
        count += 1

  #Variable still called train_cls_head, but is average of every words embedding across sentence
  train_cls_head = torch.FloatTensor(train_word_embed_eachinstance)
  print("Number of Bert Embeddings created: {}\t\t Should equal number of training instances".format(len(train_cls_head)))
  print("Length of a single BERT embedding: {}\t\t Should be 768".format(len(train_cls_head[0])))

  print("Bert Vectors Created for all Train instances")

Get BERT embeddings for the test instances

In [178]:
if BERT:
  test_word_embed_eachinstance = []
  count = 0
  print("There will be {} batches".format(len(validation_dataloader)))
  for batch in validation_dataloader:
    if count%5==0 and count!=0:
      print("On batch number {}".format(count))
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    t_input_ids, t_input_mask, b_labels = batch
    
    # Telling the model_bert not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass
        outputs = model(t_input_ids, token_type_ids=None, attention_mask=t_input_mask)
        
        for sentence in outputs[0]:
          #print(len(sentence))
          sentence_embed = []
          for embed_index in range(len(word)):
            index_total = 0
            for word in sentence:
              index_total += word[embed_index]
            sentence_embed.append(index_total / len(word))
          test_word_embed_eachinstance.append(sentence_embed)
        
        count += 1


  test_cls_head = torch.FloatTensor(test_word_embed_eachinstance)
  print("Number of Bert Embeddings created: {}\t\t Should equal number of test instances".format(len(test_cls_head)))
  print("Length of a single BERT embedding: {}\t\t Should be 768".format(len(test_cls_head[0])))
  print("Bert Vectors Embeddings Created for all Test instances")

In This block, Will combine the vectors produced by tfidf with the tensors/vectors created by BERT

In [179]:
train_combined_vectors = []
test_combined_vectors = []

#IF doing both BERT and Tf-IDF
if TF_IDF and BERT: 
  #Turn BERT tensor to numpy array for easier work
  #train_cls_head.numpy() 
  print("TFIDF and BERT Results Same Length: {}".format(len(train_tfidf) == len(train_cls_head)))
  #Check that these are same length, if they arent, something went wrong

  if len(train_tfidf) != len(train_cls_head):
    print("Train Vectors are not the same Length, errors have occured.\n Quitting Process")
  if len(test_tfidf) != len(test_cls_head):
    
    print("Train Vectors are not the same Length, errors have occured.\n Quitting Process")

  #Combine Train TFIDF Vector into 1 vector
  for i in range(len(train_tfidf)):
    tfidf_cur_index = train_tfidf[i]
    bert_cur_index = train_cls_head[i]
    # print(len(tfidf_cur_index))
    # print(len(bert_cur_index))
    combined_entry = np.concatenate((tfidf_cur_index, bert_cur_index))
    train_combined_vectors.append(combined_entry)
  print("Train TFIDF and Combined Vectors Created")

#Combine Train TFIDF Vector into 1 vector
  for i in range(len(test_tfidf)):
    tfidf_cur_index = test_tfidf[i]
    bert_cur_index = test_cls_head[i]
    # print(len(tfidf_cur_index))
    # print(len(bert_cur_index))
    combined_entry = np.concatenate((tfidf_cur_index, bert_cur_index))
    test_combined_vectors.append(combined_entry)
  print("Test TFIDF and Combined Vectors Created")


# If BERt is not selected:
elif TF_IDF and not BERT:
  train_cls_head = [[]]
  test_cls_head = [[]]
  for i in range(len(train_tfidf)):
    tfidf_cur_index = train_tfidf[i]
    train_combined_vectors.append(tfidf_cur_index)
  for i in range(len(test_tfidf)):
    tfidf_cur_index = test_tfidf[i]
    test_combined_vectors.append(tfidf_cur_index)

#IF ONLY Bert is selected
elif BERT and not TF_IDF:
  train_tfidf = [[]]
  test_tfidf = [[]]
  for i in range(len(train_cls_head)):
    bert_cur_index = train_cls_head[i].numpy()
    train_combined_vectors.append(bert_cur_index)
  for i in range(len(test_cls_head)):
    bert_cur_index = test_cls_head[i].numpy()
    test_combined_vectors.append(bert_cur_index)

print("Expected Length is Bert vector {} + TFIDF vector {} for a total length of {}".format(len(train_cls_head[0]), len(train_tfidf[0]), (len(train_tfidf[0])+len(train_cls_head[0]))))
print("Combined Train Vector is {} Long".format(len(train_combined_vectors[0])))
print("Combined Test Vector is {} Long".format(len(test_combined_vectors[0])))

Expected Length is Bert vector 0 + TFIDF vector 768 for a total length of 768
Combined Train Vector is 768 Long
Combined Test Vector is 768 Long


Neural Network Training and Testing

In [180]:
#Create training loader for neural netowrk
train_combined_vect_nn = torch.tensor(train_combined_vectors).squeeze()
train_label_tensor_nn = torch.tensor(train_labels)


train_data_nn = TensorDataset(train_combined_vect_nn, train_label_tensor_nn)
train_sampler_nn = RandomSampler(train_data_nn)
train_dataloader_nn = DataLoader(train_data_nn, sampler=train_sampler_nn, batch_size = batch_size_nn)
print("Train Loader Created")

#BUILD NEURAL NETWORK WITH SOFTMAX CLASSICATION OUTPUT
#NUMBER OF INPUTS = length of combined vectors
num_inputs = len(train_combined_vectors[0]) 

# Build a feed-forward network
# one_third_cut = int(num_inputs/3)
# nn_model = nn.Sequential(nn.Linear(num_inputs, num_inputs),
#                          nn.ReLU(),
#                          nn.Linear(num_inputs, one_third_cut *2),
#                          nn.ReLU(),
#                          nn.Linear(one_third_cut *2, int(2* one_third_cut / 3 )),
#                          nn.ReLU(),
#                          nn.Linear(int(2* one_third_cut / 3 ), 1),
#                          nn.Sigmoid())

# nn_model = nn.Sequential(nn.Linear(num_inputs, int(num_inputs/2)),
#                          nn.ReLU(),
#                          nn.Linear(int(num_inputs/2), 1),
#                          nn.Sigmoid())


nn_model = nn.Sequential(nn.Linear(num_inputs, 1),
                           nn.Sigmoid())
 
print(nn_model)

criterion = nn.BCELoss()

optimizer = optim.Adam(nn_model.parameters(), lr = learning_rate)

# Train the Neural Network
nn_model.train()

for e in range(num_epochs):
    train_pred, train_act = [], []
    print("Epoch {}".format(e))
    running_loss = 0
    nn_model.train()
    for step, batch in enumerate(train_dataloader_nn):
        # if step%10==0:
        #  print("Step: " + str(step))
        batch = tuple(t.to(device) for t in batch)
        vectors, labels = batch
        # Training pass
        optimizer.zero_grad()
        
        output = nn_model(vectors.float()
        )
        for index in range(len(output)):
          if output[index] > 0.5:
            train_pred.append(1)
            train_act.append(labels[index])
          else:
            train_pred.append(0)
            train_act.append(labels[index])

        labels = labels.unsqueeze(1).float()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        correct = 0
        for i in range(len(train_pred)):
          if train_pred[i] == train_act[i]:
            correct +=1
        print(f"Average Training loss Over The Epoch: {running_loss/len(train_dataloader_nn)}")
        print("Epoch Accuracy was {}".format(correct / len(train_pred)))
  


Train Loader Created
Sequential(
  (0): Linear(in_features=768, out_features=1, bias=True)
  (1): Sigmoid()
)
Epoch 0
Average Training loss Over The Epoch: 0.6701675155009442
Epoch Accuracy was 0.6651982378854625
Epoch 1
Average Training loss Over The Epoch: 0.6164062882375996
Epoch Accuracy was 0.7679882525697503
Epoch 2
Average Training loss Over The Epoch: 0.5774532392708182
Epoch Accuracy was 0.7738619676945668
Epoch 3
Average Training loss Over The Epoch: 0.5460939247008653
Epoch Accuracy was 0.7856093979441997
Epoch 4
Average Training loss Over The Epoch: 0.5207155604808651
Epoch Accuracy was 0.7936857562408223
Epoch 5
Average Training loss Over The Epoch: 0.5009981040020435
Epoch Accuracy was 0.8091042584434655
Epoch 6
Average Training loss Over The Epoch: 0.4835825344623878
Epoch Accuracy was 0.8113069016152716
Epoch 7
Average Training loss Over The Epoch: 0.4673906817422276
Epoch Accuracy was 0.8237885462555066
Epoch 8
Average Training loss Over The Epoch: 0.4543924892854969
E

Test the Neural Network

In [181]:
#Create Test loader for neural netowrk
test_combined_vect_nn = torch.tensor(test_combined_vectors).squeeze()
test_label_tensor_nn = torch.tensor(test_labels)

# print(test_combined_vect_nn.size())
# print(test_label_tensor_nn.size())

test_data_nn = TensorDataset(test_combined_vect_nn, test_label_tensor_nn)
test_sampler_nn = RandomSampler(test_data_nn)
test_dataloader_nn = DataLoader(test_data_nn, sampler=test_sampler_nn, batch_size = 1)
print("length dataloader {}".format(len(test_dataloader_nn)))
# Tracking variables for Neural Network Test (MODIFY TO CHANGE WHAT IS REPORTED)


predictions, true_labels = [], []

count=0
# Evaluate data for one epoch
with torch.no_grad():
  nn_model.eval()
  for batch in test_dataloader_nn:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    t_vectors, t_labels = batch

    output = nn_model(t_vectors.float())
    
    if output > 0.5:
      predictions.append(1)
      true_labels.append(t_labels.item())
      #print(1)
    else:
      predictions.append(0)
      true_labels.append(t_labels.item())
      #print(0)

    
    count+=1

print("Testing complete!")
true_pos, true_neg, false_pos, false_neg = 0, 0, 0, 0

for i in range(len(predictions)):
  if predictions[i] == true_labels[i] and predictions[i] == 1:
    true_pos +=1
  if predictions[i] == true_labels[i] and predictions[i] == 0:
    true_neg +=1
  if predictions[i] != true_labels[i] and predictions[i] == 1:
    false_pos += 1
  if predictions[i] != true_labels[i] and predictions[i] == 0:
    false_neg +=1

print("Accuracy Equals \t{}".format((true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)))
precision = true_pos/(true_pos+false_pos)
print("Precision Equals \t{}".format(precision))
recall = true_pos/(true_pos+false_neg)
print("Recall Equals\t{}".format(recall))
print("F-Score Equals \t{}".format(2*( (precision * recall) / (precision + recall) ) ) )

print((true_pos+true_neg+false_pos+false_neg))
print(len(predictions))


length dataloader 342
Testing complete!
Accuracy Equals 	0.6842105263157895
Precision Equals 	0.6740331491712708
Recall Equals	0.7134502923976608
F-Score Equals 	0.6931818181818181
342
342
